##### Imports

In [ ]:
# !pip install wandb

In [14]:
import json
import pandas as pd

##### Optional Utilities

In [16]:
from wandb import Run

# Needs a run object from wandb
def get_metadata_from_run(run: Run):
    with run.file("wandb-metadata.json").download(root=".", replace=True) as f:
        data = json.load(f)
    return data

# Save record to json
def save_to_json(records: [], filename: str, indent: int = 2):
    with open(f"{filename}.json", "w") as f:
        json.dump(records, f, indent=indent)

# Help func, prob should have just inlined it
def last_row(history_list):
    return history_list[-1] if history_list else {}

#### Collect from WandB api and build df

In [2]:
# Insert post weblink should follow scheme org/project/id
sweep_path = "example-org-name/example-project-name/3d94i2u8"  # end with id of run

In [ ]:
# Use wandb api
from wandb.apis.public import Api
sweep = Api().sweep(sweep_path)

In [ ]:
# Collect runs from sweep
runs = sweep.runs

In [12]:
# Create record array of sweep info | Takes a second if there is a lot of runs
records = []
for run in runs:
    rec = {
        "run_id": run.id,
        "name": run.name,
        "url": run.url,
        "config": dict(run.config),
        "history": []
    }

    for row in run.scan_history():
        rec["history"].append(row)

    records.append(rec)

In [15]:
# Test a view of a runs history
records[0]['history']

[{'_timestamp': 1757369814.310271,
  'train_loss': 1.987232540879408,
  'test_accuracy': 31.02,
  'train_accuracy': 26.996,
  '_step': 0,
  'epoch': 1,
  '_runtime': 13.5769879,
  'test_loss': 1.856789961741988},
 {'_timestamp': 1757369826.1602318,
  'train_loss': 1.7675872199675615,
  'test_accuracy': 38.18,
  'train_accuracy': 35.854,
  '_step': 1,
  'epoch': 2,
  '_runtime': 25.4264448,
  'test_loss': 1.6927686565241236},
 {'_timestamp': 1757369838.4953878,
  'train_loss': 1.6731136834529965,
  'test_accuracy': 40.66,
  'train_accuracy': 38.844,
  '_step': 2,
  'epoch': 3,
  '_runtime': 37.761473,
  'test_loss': 1.6367508210953634},
 {'_timestamp': 1757369851.8818395,
  'train_loss': 1.6222170637086835,
  'test_accuracy': 41.89,
  'train_accuracy': 41.156,
  '_step': 3,
  'epoch': 4,
  '_runtime': 51.1481195,
  'test_loss': 1.6045109367674324},
 {'_timestamp': 1757369865.4788415,
  'train_loss': 1.5843330402203533,
  'test_accuracy': 43.05,
  'train_accuracy': 42.878,
  '_step': 4,


In [22]:
# Build some dfs
summary_rows = []
hist_rows = []

for r in records:
    cfg   = r.get("config", {})
    hist  = r.get("history", [])
    last  = last_row(hist)

    # summary row
    summary_rows.append({
        "run_id": r["run_id"],
        "name": r.get("name"),
        "url": r.get("url"),
        **{str(k): v for k, v in cfg.items()},
        **{str(k): v for k, v in last.items()},
    })

    # long history rows
    for h in hist:
        hist_rows.append({
            "run_id": r["run_id"],
            "name": r.get("name"),
            "url": r.get("url"),
            **{str(k): v for k, v in cfg.items()},
            **h,  # includes epoch, train_loss, test_accuracy, etc.
        })

summary_df = pd.DataFrame(summary_rows)
hist_df    = pd.DataFrame(hist_rows)

# Create Cols
summary_df["runtime_min"] = summary_df["_runtime"] / 60
hist_df["runtime_min"] = hist_df["_runtime"] / 60

# Some Cleaning
if "_timestamp" in summary_df.columns:
    summary_df["_timestamp"] = pd.to_datetime(summary_df["_timestamp"], unit="s")
if "_timestamp" in hist_df.columns:
    hist_df["_timestamp"] = pd.to_datetime(hist_df["_timestamp"], unit="s")

In [23]:
# To CSV
summary_df.to_csv("summary.csv", index=False)
hist_df.to_csv("hist.csv", index=False)

,run_id,name,url,seed,epochs,stride,dropout,padding,activation,batch_size,...,apply_dropout_at,_timestamp,train_loss,test_accuracy,train_accuracy,_step,epoch,_runtime,test_loss,runtime_min
0,pz9yom7x,peach-sweep-1,https://wandb.ai/marcocassar-belmont-universit...,42,30,2,0.2,1,ReLU,64,...,2,2025-09-08 22:23:24.570344448,1.220150,56.27,56.812,29,30,403.836469,1.236209,6.730608
1,ryizrmb9,lyric-sweep-2,https://wandb.ai/marcocassar-belmont-universit...,42,30,2,0.2,1,ReLU,64,...,2,2025-09-08 22:21:02.065508842,0.782321,67.76,72.464,29,30,230.265375,0.926623,3.837756
2,cp4wz5qy,stoic-sweep-3,https://wandb.ai/marcocassar-belmont-universit...,42,30,2,0.2,1,ReLU,64,...,2,2025-09-08 22:21:29.933848858,0.325412,72.46,88.088,29,30,243.161724,0.927018,4.052695
3,9vf4nu89,denim-sweep-4,https://wandb.ai/marcocassar-belmont-universit...,42,30,2,0.2,1,ReLU,64,...,2,2025-09-08 22:21:22.830347776,1.098383,59.89,61.308,29,30,226.840999,1.127277,3.780683
4,29lnmznm,zany-sweep-5,https://wandb.ai/marcocassar-belmont-universit...,42,30,2,0.2,1,ReLU,64,...,2,2025-09-08 22:21:57.266728878,0.550354,69.19,80.406,29,30,233.702599,0.957070,3.895043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,l47wyxir,atomic-sweep-284,https://wandb.ai/marcocassar-belmont-universit...,42,30,2,0.3,1,SiLU,128,...,2,2025-09-09 02:02:05.525975227,0.579243,67.51,79.506,29,30,213.508436,0.947682,3.558474
284,5c3jwjju,happy-sweep-285,https://wandb.ai/marcocassar-belmont-universit...,42,30,2,0.3,1,SiLU,128,...,2,2025-09-09 02:02:26.379442215,0.292240,72.49,89.710,29,30,219.684882,1.002373,3.661415
285,pqkdch78,fluent-sweep-286,https://wandb.ai/marcocassar-belmont-universit...,42,30,2,0.3,1,SiLU,128,...,2,2025-09-09 02:03:24.150870323,0.961231,62.35,66.254,29,30,195.716253,1.060411,3.261938
286,wakffzj1,atomic-sweep-287,https://wandb.ai/marcocassar-belmont-universit...,42,30,2,0.3,1,SiLU,128,...,2,2025-09-09 02:07:07.282821655,0.310411,67.35,88.828,29,30,371.714876,1.148178,6.195248
